## Travel chatbot

1. Create a bot with the Microsoft Bot Framework. 
2. Deploy it to the Web App Service (no Azure bot).
3. Connect the bot to the Azure Cognitive Service LUIS for natural language processing.
4. Train the LUIS model using the data.
5. Use the aiohttp library to create a HTTP server for the bot to run on.
6. Test the bot using the Bot Framework Emulator.

In [1]:
import pandas as pd
import json
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_json("frames/frames.json")
df.head()

,user_id,turns,wizard_id,id,labels
0,U22HTHYNP,[{'text': 'I'd like to book a trip to Atlantis...,U21DKG18C,e2c0fc6c-2134-4891-8353-ef16d8412c9a,"{'userSurveyRating': 4.0, 'wizardSurveyTaskSuc..."
1,U21E41CQP,"[{'text': 'Hello, I am looking to book a vacat...",U21DMV0KA,4a3bfa39-2c22-42c8-8694-32b4e34415e9,"{'userSurveyRating': 3.0, 'wizardSurveyTaskSuc..."
2,U21RP4FCY,[{'text': 'Hello there i am looking to go on a...,U21E0179B,6e67ed28-e94c-4fab-96b6-68569a92682f,"{'userSurveyRating': 2.0, 'wizardSurveyTaskSuc..."
3,U22HTHYNP,[{'text': 'Hi I'd like to go to Caprica from B...,U21DKG18C,5ae76e50-5b48-4166-9f6d-67aaabd7bcaa,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."
4,U21E41CQP,"[{'text': 'Hello, I am looking to book a trip ...",U21DMV0KA,24603086-bb53-431e-a0d8-1dcc63518ba9,"{'userSurveyRating': 5.0, 'wizardSurveyTaskSuc..."


In [64]:
# get the 'frames' data from json of cell: 
user = 0
print(df.iloc[user]['turns'][0]['text'])
df.iloc[user]['turns'][0]['labels']['acts']

I'd like to book a trip to Atlantis from Caprica on Saturday, August 13, 2016 for 8 adults. I have a tight budget of 1700.


[{'args': [{'val': 'book', 'key': 'intent'}], 'name': 'inform'},
 {'args': [{'val': 'Atlantis', 'key': 'dst_city'},
   {'val': 'Caprica', 'key': 'or_city'},
   {'val': 'Saturday, August 13, 2016', 'key': 'str_date'},
   {'val': '8', 'key': 'n_adults'},
   {'val': '1700', 'key': 'budget'}],
  'name': 'inform'}]

In [65]:
df.shape

(1369, 5)

### Convert frames data to Luis compatible data

In [3]:
luis_entities = [
    "or_city",
    "dst_city",
    "str_date",
    "end_date",
    "budget"
]

def get_user_requests(data, index):

    json_part = {}
    txt = data.iloc[index]['turns'][0]["text"].lower()
    json_part["text"] = txt
    json_part["intentName"] = "BookFlight"

    entities = []

    for act in data.iloc[index]['turns'][0]['labels']['acts']:
        
        for arg in act['args']:
            if arg['key'] in luis_entities:
                entity = {}
                key = arg['key'].lower()
                if 'val' in arg.keys():
                    val = arg['val'].lower()
                    if val != '-1':
                        startPos = txt.index(val)
                        endPos = startPos + len(val)
                        entity['entityName'] = key
                        entity['startCharIndex'] = startPos
                        entity['endCharIndex'] = endPos
                        entities.append(entity)

    json_part['entityLabels'] = entities

    return json_part


get_user_requests(df, 1)

{'text': 'hello, i am looking to book a vacation from gotham city to mos eisley for $2100.',
 'intentName': 'BookFlight',
 'entityLabels': [{'entityName': 'dst_city',
   'startCharIndex': 59,
   'endCharIndex': 69},
  {'entityName': 'or_city', 'startCharIndex': 44, 'endCharIndex': 55},
  {'entityName': 'budget', 'startCharIndex': 75, 'endCharIndex': 79}]}

In [4]:
def convert_luis(data):
    luis_data = []
    for i in range(data.shape[0]):
        json_part = get_user_requests(data, i)
        luis_data.append(json_part)
    return luis_data

luis_data = convert_luis(df)

In [5]:
print("Length luis_data: " + str(len(luis_data)))
luis_data

Length luis_data: 1369


[{'text': "i'd like to book a trip to atlantis from caprica on saturday, august 13, 2016 for 8 adults. i have a tight budget of 1700.",
  'intentName': 'BookFlight',
  'entityLabels': [{'entityName': 'dst_city',
    'startCharIndex': 27,
    'endCharIndex': 35},
   {'entityName': 'or_city', 'startCharIndex': 41, 'endCharIndex': 48},
   {'entityName': 'str_date', 'startCharIndex': 52, 'endCharIndex': 77},
   {'entityName': 'budget', 'startCharIndex': 117, 'endCharIndex': 121}]},
 {'text': 'hello, i am looking to book a vacation from gotham city to mos eisley for $2100.',
  'intentName': 'BookFlight',
  'entityLabels': [{'entityName': 'dst_city',
    'startCharIndex': 59,
    'endCharIndex': 69},
   {'entityName': 'or_city', 'startCharIndex': 44, 'endCharIndex': 55},
   {'entityName': 'budget', 'startCharIndex': 75, 'endCharIndex': 79}]},
 {'text': 'hello there i am looking to go on a vacation with my family to gotham city, can you help me?',
  'intentName': 'BookFlight',
  'entityLabels

In [6]:
train, test = train_test_split(luis_data, test_size=0.3, random_state=42)

In [7]:
print("Length train: " + str(len(train)))
print("Length test: " + str(len(test)))

Length train: 958
Length test: 411


In [8]:
with open("luis_data.json", "w") as outfile:
    json.dump(luis_data, outfile)

with open("train.json", "w") as outfile:
    json.dump(train, outfile)

with open("test.json", "w") as outfile:
    json.dump(test, outfile)